In [ ]:
pip install ultralytics opencv-python numpy torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose Model
model = YOLO("yolov8n-pose.pt")

# Load the video
video_path = "/content/cow mounting _vedio.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
output_path = "/content/Cow_Heat_Detection.mp4"
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Function to check if cow is mounting another cow
def is_mounted(cow1, cow2):
    """
    Checks if one cow is on top of another (mounting behavior).
    - cow1: Keypoints of first cow
    - cow2: Keypoints of second cow
    """
    try:
        back1_y = cow1[8][1]  # Back keypoint of cow1
        front_leg1_y = cow1[10][1]  # Front leg of cow1

        back2_y = cow2[8][1]  # Back keypoint of cow2
        front_leg2_y = cow2[10][1]  # Front leg of cow2

        # Condition: If back of cow1 is much higher than back of cow2 -> Mounting
        if back1_y < back2_y - 40 and front_leg1_y < front_leg2_y - 40:
            return True
        return False
    except:
        return False

# Process video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO Pose detection
    results = model(frame)

    cows = []
    for result in results:
        if result.keypoints is None or len(result.keypoints.xy) == 0:
            continue  # Skip if no keypoints detected

        for keypoints in result.keypoints.xy.cpu().numpy():
            if len(keypoints) < 13:  # Ensure enough keypoints detected
                continue
            cows.append(keypoints)

    # Check for mounting behavior
    in_heat_count = 0
    for i, cow1 in enumerate(cows):
        for j, cow2 in enumerate(cows):
            if i != j and is_mounted(cow1, cow2):
                in_heat_count += 1
                x, y = int(cow1[8][0]), int(cow1[8][1])
                cv2.putText(frame, "In Heat", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Display mounting count
    cv2.putText(frame, f"Cows in Heat: {in_heat_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Write frame to output video
    out.write(frame)

cap.release()
out.release()

print("Processing complete! Download the processed video here:")
print(output_path)



0: 384x640 (no detections), 170.5ms
Speed: 6.0ms preprocess, 170.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 169.9ms
Speed: 5.5ms preprocess, 169.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 164.1ms
Speed: 6.4ms preprocess, 164.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 166.8ms
Speed: 5.8ms preprocess, 166.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 180.6ms
Speed: 6.2ms preprocess, 180.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 164.1ms
Speed: 5.7ms preprocess, 164.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 179.4ms
Speed: 5.8ms preprocess, 179.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 162.3ms
Speed: 4.8ms prepr

In [ ]:
from IPython.display import display, HTML

video_path = "/content/Cow_Heat_Detection.mp4"
display(HTML(f"""
    <video width="640" height="480" controls>
        <source src="{video_path}" type="video/mp4">
    </video>
"""))


In [ ]:
from google.colab import files
files.download("/content/Cow_Heat_Detection.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>